<a href="https://colab.research.google.com/github/goelpravin/practice/blob/master/Cnnscratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) Get Data 
2) Unzip if zipped
3) Split data between training and testing including on directories

In [1]:
#get cats versus dogs data

!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
    -O "/tmp/cats-and-dogs.zip"

--2020-07-29 14:19:22--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.204.71.31, 2600:1407:f800:4a5::e59, 2600:1407:f800:49b::e59
Connecting to download.microsoft.com (download.microsoft.com)|23.204.71.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘/tmp/cats-and-dogs.zip’

/tmp/cats-and-dogs. 100%[===================>] 786.68M  68.8MB/s    in 12s     

2020-07-29 14:19:34 (64.9 MB/s) - ‘/tmp/cats-and-dogs.zip’ saved [824894548/824894548]



In [2]:
#Unzip the downloaded zip file and check how many files got extracted for each sub-dir
import zipfile
import os
local_zip = '/tmp/cats-and-dogs.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()
print(len(os.listdir('/tmp/PetImages/Cat/')))
print(len(os.listdir('/tmp/PetImages/Dog/')))

12501
12501


In [3]:
import random
from shutil import copyfile
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    source_files = os.listdir(SOURCE);
    random.sample(source_files, len(source_files))
    files_attempted_to_move = 0
    files_to_move_to_train = SPLIT_SIZE*len(source_files)
    #first empty both training and testing folders
    training_files = os.listdir(TRAINING)
    for t_file in training_files:
        os.remove(os.path.join(TRAINING, t_file))
    testing_files = os.listdir(TESTING)
    for tst_file in testing_files:
        os.remove(os.path.join(TRAINING, tst_file))
    for src_file in source_files:
        src_file_full_path = os.path.join(SOURCE,src_file)
        training_file_fpath = os.path.join(TRAINING, src_file)
        testing_file_fpath = os.path.join(TESTING, src_file)
        if(os.path.getsize(src_file_full_path)>0):
            if(files_attempted_to_move<files_to_move_to_train):
                copyfile(src_file_full_path, training_file_fpath)
            else:
                copyfile(src_file_full_path, testing_file_fpath)
            files_attempted_to_move = files_attempted_to_move + 1

In [4]:
def makeDirsPassOSError(path):
        try:
            os.makedirs(path)
        except OSError:
            pass

In [5]:
def ensureAllDirs():
  parentDir = "/tmp/cats-v-dogs/"
  makeDirsPassOSError(parentDir)
  training_path = os.path.join(parentDir, "training")
  testing_path = os.path.join(parentDir, "testing")
  makeDirsPassOSError(training_path)
  makeDirsPassOSError(testing_path)
  training_cats_path = os.path.join(training_path,"cats")
  training_dogs_path = os.path.join(training_path,"dogs")
  makeDirsPassOSError(training_cats_path)
  makeDirsPassOSError(training_dogs_path)
  testing_cats_path = os.path.join(testing_path,"cats")
  testing_dogs_path= os.path.join(testing_path,"dogs")
  makeDirsPassOSError(testing_cats_path)
  makeDirsPassOSError(testing_dogs_path)

In [6]:
ensureAllDirs()
CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/tmp/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/tmp/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)
print(len(os.listdir('/tmp/cats-v-dogs/training/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/training/dogs/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/dogs/')))

11251
11251
1249
1249


In [7]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('cats') and 1 for the other ('dogs')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

In [8]:
#create generators
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1/255)
batch_size_training = 1280
batch_size_validation = 130
# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/tmp/cats-v-dogs/training/',  
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=batch_size_training,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        '/tmp/cats-v-dogs/testing/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=batch_size_validation,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 22502 images belonging to 2 classes.
Found 2496 images belonging to 2 classes.


In [ ]:

num_training_images = len(os.listdir('/tmp/cats-v-dogs/training/'))
epoch_steps = num_training_images/batch_size_training
num_testing_images = len(os.listdir('/tmp/cats-v-dogs/testing/'))
validation_steps = num_testing_images/batch_size_validation
history = model.fit(
      train_generator,
      steps_per_epoch=epoch_steps,  
      epochs=5,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=validation_steps)

Epoch 1/5
